In [295]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

import warnings
warnings.filterwarnings('ignore')

In [296]:
data = pd.read_csv('books.csv')

In [297]:
print(data.shape)
print(f'Количество пропусков в данных: {data.isna().sum().sum()}')
print(f'Доля пропусков в данных: {round(data.isna().sum().sum() / (data.shape[0]*data.shape[1]), 2)}')

(3600, 23)
Количество пропусков в данных: 16962
Доля пропусков в данных: 0.2


In [298]:
print(f'Всего объектов у каждого признака (если нет пропусков): {data.shape[0]}')
data.isna().sum().sort_values(ascending=False)

Всего объектов у каждого признака (если нет пропусков): 3600


Разделы комикса           3599
Тип комикса               3599
Художник                  3592
Переводчик                3383
Издательский бренд        2203
Издательство                81
old_price                   80
Тип обложки                 59
cnt_reviews                 50
avg_rating                  50
Тираж                       50
Количество страниц          45
current_price               42
ISBN                        29
Серия                       25
author                      24
Возрастные ограничения      23
Год издания                 18
Вес, г                       2
Размер                       2
ID товара                    2
title                        2
link                         2
dtype: int64

In [299]:
data['cnt_reviews'].fillna('0 оценок', inplace=True)

In [300]:
good_columns = []
bad_columns = []
for column in data.columns:
    all_items = data.shape[0]
    if data.isna().sum().sort_values(ascending=False)[column] / all_items > 0.3:
        bad_columns.append(column)
    else:
        good_columns.append(column)
        
print(f'В признаках {bad_columns} недостаточно объектов, их стоит удалить.')  
print()
print(f'После удаления всех "плохих признаков" остались следующие: {good_columns}')

В признаках ['Издательский бренд', 'Переводчик', 'Художник', 'Тип комикса', 'Разделы комикса'] недостаточно объектов, их стоит удалить.

После удаления всех "плохих признаков" остались следующие: ['link', 'current_price', 'old_price', 'avg_rating', 'cnt_reviews', 'title', 'author', 'ID товара', 'Издательство', 'Серия', 'Год издания', 'ISBN', 'Количество страниц', 'Размер', 'Тип обложки', 'Тираж', 'Вес, г', 'Возрастные ограничения']


In [301]:
data = data[good_columns]
data.dropna(inplace=True)
data = data.reset_index(drop=True)

In [302]:
data['Год издания'] = data['Год издания'].apply(lambda x: str(x).split('.')[0])

data['Год издания'] = data['Год издания'].astype(int)

### Удаляем заполненные, но ненужные признаки

In [303]:
data = data.drop(['ID товара', 'ISBN', 'Серия', 'link', 'old_price', ], axis=1)

In [304]:
print(data.shape)

print(data.columns)

(3260, 13)
Index(['current_price', 'avg_rating', 'cnt_reviews', 'title', 'author',
       'Издательство', 'Год издания', 'Количество страниц', 'Размер',
       'Тип обложки', 'Тираж', 'Вес, г', 'Возрастные ограничения'],
      dtype='object')


In [305]:
data = data.rename(columns={
    'current_price': 'price',
    'Количество страниц': 'pages_cnt',
    'Вес, г': 'weight',
    'Издательство': 'publisher',
    'Год издания': 'publication_year',
    'Тип обложки': 'cover_type',
    'Тираж': 'tirage',
    'Возрастные ограничения': 'reading_age'
    })

In [306]:
data.columns

Index(['price', 'avg_rating', 'cnt_reviews', 'title', 'author', 'publisher',
       'publication_year', 'pages_cnt', 'Размер', 'cover_type', 'tirage',
       'weight', 'reading_age'],
      dtype='object')

### Обработка конкретных признаков:

In [307]:
# надо применить OHE
print(data['cover_type'].value_counts())
print()
print(data['reading_age'].value_counts())
print()
print(data['publication_year'].value_counts())
print()
print(data['publisher'].value_counts())
print()
print(data['author'].value_counts())

cover_type
Твёрдый переплёт                  1728
Мягкий переплёт                   1531
Мягкий заламинированный картон       1
Name: count, dtype: int64

reading_age
16+    2405
12+     680
18+     150
6+       22
0+        3
Name: count, dtype: int64

publication_year
2024    1469
2025     846
2023     571
2022     265
2021      63
2011      12
2012      11
2020       6
2015       5
2018       4
2019       3
2014       2
2010       1
2005       1
2013       1
Name: count, dtype: int64

publisher
АСТ                       1343
Эксмо                     1008
Азбука                     595
Иностранка                 158
Манн, Иванов и Фербер       90
Corpus                      29
Вече                         9
Махаон                       4
Жанры                        3
Альпина Паблишер             3
Проспект                     3
Мартин                       3
Лениздат                     2
Обложка                      2
Агентство Алгоритм ООО       2
КоЛибри                      1
Д

#### Замена значений на "Другой"

In [308]:
def replace_to_other(df, column_name, threshold):

    # Получаем value_counts и фильтруем частые категории
    df[column_name] = df[column_name].astype(str)

    value_counts = df[column_name].value_counts()
    common_categories = value_counts[value_counts >= threshold].index

    df[column_name] = df[column_name].where(df[column_name].isin(common_categories), 'Другой')

In [309]:
replace_to_other(data, 'publication_year', 60)
print(data['publication_year'].value_counts())
print()

replace_to_other(data, 'publisher', 60)
print(data['publisher'].value_counts())
print()

replace_to_other(data, 'author', 50)
print(data['author'].value_counts())
print()

publication_year
2024      1469
2025       846
2023       571
2022       265
2021        63
Другой      46
Name: count, dtype: int64

publisher
АСТ                      1343
Эксмо                    1008
Азбука                    595
Иностранка                158
Манн, Иванов и Фербер      90
Другой                     66
Name: count, dtype: int64

author
Другой               2623
Федор Достоевский     129
Михаил Булгаков        77
Джек Лондон            72
Джейн Остен            68
Эрих Ремарк            67
Джордж Оруэлл          64
Лев Толстой            58
Луиза Мэй Олкотт       52
Николай Гоголь         50
Name: count, dtype: int64



**OHE**

In [310]:
def OHE_features(data):
    categorical_features = []
    for column in data.columns:
        if len(data[column].value_counts()) <= 10:
            #data[column] = data[column].astype(str)
            categorical_features.append(column)
    
    ohe = OneHotEncoder(sparse_output=False)
    ohe_data = ohe.fit_transform(data[categorical_features])
    ohe_columns = ohe.get_feature_names_out(categorical_features)
    ohe_data = pd.DataFrame(ohe_data, columns=ohe_columns)
    
    data = data.drop(categorical_features, axis=1)
    data = pd.concat([data, ohe_data], axis=1)
    
    return data

In [311]:
data = OHE_features(data)

**Размер**

In [312]:
def split_sizes(data, size_column):
    data['thickness'] = data[size_column].dropna().apply(lambda x: float(x.split('x')[0]))
    data['width'] = data[size_column].dropna().apply(lambda x: float(x.split('x')[1]))
    data['length'] = data[size_column].dropna().apply(lambda x: float(x.split('x')[2]))
    data['volume'] = data['thickness'] * data['width'] * data['length']
    
    data = data.drop([size_column], axis=1)
    
    return data

In [313]:
data = split_sizes(data, 'Размер')

**Некоторые количественные признаки**

In [314]:
# data['old_price'] = data['old_price'].apply(lambda x: x.split(' ')[0])
data['price'] = data['price'].apply(lambda x: x.split(' ')[0])

data['cnt_reviews'] = data['cnt_reviews'].apply(lambda x: x.split()[0]).astype(int)

In [315]:
data.columns

Index(['price', 'avg_rating', 'cnt_reviews', 'title', 'pages_cnt', 'tirage',
       'weight', 'author_Джейн Остен', 'author_Джек Лондон',
       'author_Джордж Оруэлл', 'author_Другой', 'author_Лев Толстой',
       'author_Луиза Мэй Олкотт', 'author_Михаил Булгаков',
       'author_Николай Гоголь', 'author_Федор Достоевский',
       'author_Эрих Ремарк', 'publisher_АСТ', 'publisher_Азбука',
       'publisher_Другой', 'publisher_Иностранка',
       'publisher_Манн, Иванов и Фербер', 'publisher_Эксмо',
       'publication_year_2021', 'publication_year_2022',
       'publication_year_2023', 'publication_year_2024',
       'publication_year_2025', 'publication_year_Другой',
       'cover_type_Мягкий заламинированный картон',
       'cover_type_Мягкий переплёт', 'cover_type_Твёрдый переплёт',
       'reading_age_0+', 'reading_age_12+', 'reading_age_16+',
       'reading_age_18+', 'reading_age_6+', 'thickness', 'width', 'length',
       'volume'],
      dtype='object')

In [316]:
int_features = ['price', 'cnt_reviews', 
                'pages_cnt', 'tirage']
float_features = ['avg_rating', 'length', 'width', 'thickness', 'weight', 'volume']

In [317]:
data[int_features] = data[int_features].astype(int)
data[float_features] = data[float_features].astype(float)

In [318]:
data.dtypes

price                                          int64
avg_rating                                   float64
cnt_reviews                                    int64
title                                         object
pages_cnt                                      int64
tirage                                         int64
weight                                       float64
author_Джейн Остен                           float64
author_Джек Лондон                           float64
author_Джордж Оруэлл                         float64
author_Другой                                float64
author_Лев Толстой                           float64
author_Луиза Мэй Олкотт                      float64
author_Михаил Булгаков                       float64
author_Николай Гоголь                        float64
author_Федор Достоевский                     float64
author_Эрих Ремарк                           float64
publisher_АСТ                                float64
publisher_Азбука                             f

# Итог:

In [319]:
data.shape

(3260, 41)

In [320]:
data.head()

,price,avg_rating,cnt_reviews,title,pages_cnt,tirage,weight,author_Джейн Остен,author_Джек Лондон,author_Джордж Оруэлл,...,cover_type_Твёрдый переплёт,reading_age_0+,reading_age_12+,reading_age_16+,reading_age_18+,reading_age_6+,thickness,width,length,volume
0,312,4.1,925,Скорбь Сатаны,512,30000,250.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,2.1,11.5,18.0,434.7
1,312,4.1,1341,1984 (новый перевод),320,30000,200.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,2.4,11.5,18.0,496.8
2,303,4.2,1288,Преступление и наказание,672,30000,319.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,2.5,17.5,18.0,787.5
3,284,4.5,363,Человек недостойный,192,25000,132.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.5,11.5,18.0,310.5
4,413,4.2,1269,Три товарища,480,30000,300.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,2.5,11.6,18.1,524.9


In [321]:
data.columns

Index(['price', 'avg_rating', 'cnt_reviews', 'title', 'pages_cnt', 'tirage',
       'weight', 'author_Джейн Остен', 'author_Джек Лондон',
       'author_Джордж Оруэлл', 'author_Другой', 'author_Лев Толстой',
       'author_Луиза Мэй Олкотт', 'author_Михаил Булгаков',
       'author_Николай Гоголь', 'author_Федор Достоевский',
       'author_Эрих Ремарк', 'publisher_АСТ', 'publisher_Азбука',
       'publisher_Другой', 'publisher_Иностранка',
       'publisher_Манн, Иванов и Фербер', 'publisher_Эксмо',
       'publication_year_2021', 'publication_year_2022',
       'publication_year_2023', 'publication_year_2024',
       'publication_year_2025', 'publication_year_Другой',
       'cover_type_Мягкий заламинированный картон',
       'cover_type_Мягкий переплёт', 'cover_type_Твёрдый переплёт',
       'reading_age_0+', 'reading_age_12+', 'reading_age_16+',
       'reading_age_18+', 'reading_age_6+', 'thickness', 'width', 'length',
       'volume'],
      dtype='object')

### Дропаем лишние OHE, чтобы избежать линейной зависимости

In [322]:
# data = data.drop(['author_Другой', 'publisher_Другой', 'publication_year_Другой', 
#                   'cover_type_Мягкий заламинированный картон', 'reading_age_6+'], axis=1)

In [323]:
data.columns

Index(['price', 'avg_rating', 'cnt_reviews', 'title', 'pages_cnt', 'tirage',
       'weight', 'author_Джейн Остен', 'author_Джек Лондон',
       'author_Джордж Оруэлл', 'author_Другой', 'author_Лев Толстой',
       'author_Луиза Мэй Олкотт', 'author_Михаил Булгаков',
       'author_Николай Гоголь', 'author_Федор Достоевский',
       'author_Эрих Ремарк', 'publisher_АСТ', 'publisher_Азбука',
       'publisher_Другой', 'publisher_Иностранка',
       'publisher_Манн, Иванов и Фербер', 'publisher_Эксмо',
       'publication_year_2021', 'publication_year_2022',
       'publication_year_2023', 'publication_year_2024',
       'publication_year_2025', 'publication_year_Другой',
       'cover_type_Мягкий заламинированный картон',
       'cover_type_Мягкий переплёт', 'cover_type_Твёрдый переплёт',
       'reading_age_0+', 'reading_age_12+', 'reading_age_16+',
       'reading_age_18+', 'reading_age_6+', 'thickness', 'width', 'length',
       'volume'],
      dtype='object')

In [327]:
data.to_csv('data_after_processing.csv', encoding='utf-8', index=False)